In [7]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np
import cv2
from superpose3d import Superpose3D

PIPELINE_ROOT = Path('../src').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama

/home/eddyod/programming/preprocessing-pipeline/src


In [37]:
def brain_to_atlas_transform(brain_coord, r, t):
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    atlas_coord = r @ brain_coord + t
    return atlas_coord.T[0] # Convert back to a row vector

def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))
        
def plot_point_sets_3d(point_sets):
    df = pd.DataFrame()
    for data, label in point_sets:
        df_cur = pd.DataFrame(data.T, columns=['x', 'y', 'z'])
        df_cur['label'] = label
        #df = df.concat(df_cur, ignore_index=True)
        df = pd.concat([df, df_cur], axis= 0)
    
    fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', width=800, height=600)
    fig.update_layout(
    scene={
        'xaxis': {'range': (0, 528)}, 
        'yaxis': {'range': (0, 320)},
        'zaxis': {'range': (0, 456)},
    })
    return fig
        

In [2]:
animal = 'Atlas'
brain = BrainStructureManager(animal)
brain.fixed_brain = BrainStructureManager('Allen')

In [ ]:
brainstem_keys = set(brain.allen_structures_keys) - brain.midbrain_keys 

In [3]:
moving_coms = brain.get_coms(annotator_id=1)
allen_coms = brain.fixed_brain.get_coms(annotator_id=1)
common_keys = sorted(allen_coms.keys() & moving_coms.keys())
allen_points = np.array([allen_coms[s] for s in common_keys])
moving_points = np.array([moving_coms[s] for s in common_keys])

In [5]:
common_midbrain_keys = sorted(allen_coms.keys() & moving_coms.keys() & brain.midbrain_keys)
allen_midbrain_points = np.array([allen_coms[s] for s in common_midbrain_keys])
moving_midbrain_points = np.array([moving_coms[s] for s in common_midbrain_keys])

In [4]:
allen_point_dict = {s:allen_coms[s] for s in common_keys}
moving_point_dict = {s:moving_coms[s] for s in common_keys}

In [5]:
len(allen_point_dict)

37

In [ ]:
distances = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    #print(atlas_point, allen_point)
    d = calculate_distance(allen_point, moving_point)
    distances.append(d)
    sortme[structure] = d
print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    print(f'{structure} distance from Allen={round(d,2)} micrometers')

In [ ]:
plot_point_sets_3d([
    (moving_points.T/25, 'unaligned moving centers'),
    (allen_points.T/25, 'Allen centers')
])

In [8]:
(RMSD, R1, t1, _) = Superpose3D(allen_points, moving_points, allow_rescale=True)
t1 = t1.reshape(3,1)
print(RMSD)
print(R1)
print(t1)

305.69115497574603
[[ 0.99841862  0.05440934  0.01413803]
 [-0.05530431  0.99576765  0.07340454]
 [-0.0100843  -0.07407036  0.99720203]]
[[-3368.22446259]
 [-2445.98352834]
 [ 1365.58366407]]


In [ ]:
allen_midbrain_points.shape

In [9]:
R, t = umeyama(moving_points.T, allen_points.T)

In [20]:
t

array([[-3368.22446259],
       [-2445.98352834],
       [ 1365.58366407]])

In [14]:
n, a1, a2 = cv2.estimateAffine3D(src=moving_points, dst=allen_points)
a2.shape


(37, 1)

In [29]:
R3 = a1[:,0:3]
t3 = a1[:,-1].reshape(3,1)

In [30]:
t3.shape

(3, 1)

In [31]:
reg_points = R @ moving_points.T + t

In [32]:
reg_points3 = R3 @ moving_points.T + t3

 That does not preclude starting the new idea but is just to say that somehow we need to get David's attention on what you have done- do a graph
= box plot of the midbrain com distances vs allen (active atlas based)
=box plot of the brainstem vs allen distances (but not using midbrain values to improve the distances)
This should provide an easy visual...


When the tethering is done for the brainstem structures though- he does not want the midbrain metrics to be included ( I don't know though if he means that the alignment is done with midbrain and then the evaluated outputs are only for the brainstem structures...?) your guess is best
So- in any case it was clear that he did not want metrics for midbrain and brainstem structures to be combined.

In [34]:
print('Mean')
print(np.mean(allen_points, axis=0))
print(np.mean(reg_points, axis=1))
print(np.mean(reg_points3, axis=1))
print('Min')
print(np.min(allen_points, axis=0))
print(np.min(reg_points, axis=1))
print('Max')
print(np.max(allen_points, axis=0))
print(np.max(reg_points, axis=1))

Mean
[10463.46540541  5123.43756757  5645.72054054]
[10463.46540541  5123.43756757  5645.72054054]
[10550.77583933  5026.79981405  5667.99930089]
Min
[8354.09 2325.   3229.  ]
[8068.25945013 2524.03061144 3263.66670863]
Max
[12832.54  6990.91  8156.39]
[13297.21697261  6871.63404678  8156.27022824]


In [ ]:
reg_point_dict = {s:moving_coms[s] for s in common_keys}

In [38]:
distances = []
distances2 = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    reg_point = brain_to_atlas_transform(moving_point, R, t)
    reg_point3 = brain_to_atlas_transform(moving_point, R3, t3)
    d = calculate_distance(allen_point, reg_point)
    ds = calculate_distance(allen_point, reg_point3)
    distances2.append(ds)
    distances.append(d)
    print(f'{structure} COM={reg_point.round()} distance={round(d,2)} cv2={round(ds,2)}')
    sortme[structure] = d

print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
print(f'n={len(distances2)}, min={min(distances2)} max={max(distances2)}, mean={np.mean(distances2)}')
#ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
#for structure, d in ds.items():
#    print(f'{structure} {round(d,2)}')

3N_L COM=[9056. 3971. 5621.] distance=208.25 cv2=296.53
3N_R COM=[9095. 4021. 5897.] distance=229.6 cv2=282.76
4N_L COM=[9588. 4017. 5449.] distance=239.67 cv2=300.26
4N_R COM=[9558. 4063. 5984.] distance=291.41 cv2=250.64
5N_L COM=[10160.  5207.  4194.] distance=134.13 cv2=77.43
5N_R COM=[10096.  5155.  7184.] distance=195.83 cv2=184.3
6N_L COM=[10782.  5309.  5256.] distance=100.62 cv2=72.85
6N_R COM=[10783.  5371.  6020.] distance=172.96 cv2=126.18
7N_L COM=[10767.  6484.  4477.] distance=333.08 cv2=266.88
7N_R COM=[11367.  6578.  7057.] distance=550.8 cv2=475.5
7n_L COM=[10551.  5651.  4532.] distance=277.45 cv2=394.57
7n_R COM=[10549.  5624.  6809.] distance=238.06 cv2=325.84
AP COM=[12736.  5530.  5510.] distance=556.23 cv2=248.77
Amb_L COM=[11672.  6404.  4294.] distance=302.4 cv2=340.8
Amb_R COM=[11692.  6237.  7010.] distance=428.44 cv2=345.68
DC_L COM=[11329.  4913.  3264.] distance=112.3 cv2=392.85
DC_R COM=[11068.  4791.  8156.] distance=292.3 cv2=631.5
IC COM=[10574.  2579

In [ ]:
reg_points.shape

In [ ]:
plot_point_sets_3d([
    (reg_points/25, 'registered moving centers'),
    (allen_points.T/25, 'Allen centers')    
])